# Section 15: Analytics API - Comprehensive Testing

## 📋 Overview
This notebook provides comprehensive testing for **Section 15: Analytics** of the VeevaVault API.

Analytics API enables data analytics, reporting capabilities, metrics collection, dashboard management, performance analytics, usage statistics, and business intelligence within the Vault ecosystem.

## 🎯 Endpoints to Test

### Analytics Information (AnalyticsService)
1. **Retrieve Analytics Data** - `GET /api/{version}/analytics`
2. **Analytics Metrics** - `GET /api/{version}/analytics/metrics`
3. **Analytics Reports** - Report generation and management
4. **Analytics Configuration** - Analytics settings and configuration

### Reporting (ReportingService)
5. **Report Generation** - `POST /api/{version}/reports`
6. **Report Execution** - `GET /api/{version}/reports/{report_id}/execute`
7. **Report Results** - `GET /api/{version}/reports/{report_id}/results`
8. **Report Templates** - Pre-defined report templates

### Performance Analytics (PerformanceService)
9. **System Performance** - System performance metrics
10. **User Activity Analytics** - User behavior and activity patterns
11. **Document Analytics** - Document usage and access patterns
12. **API Usage Analytics** - API call statistics and performance

### Dashboard Management (DashboardService)
13. **Dashboard Configuration** - `GET /api/{version}/dashboards`
14. **Dashboard Data** - `GET /api/{version}/dashboards/{dashboard_id}/data`
15. **Widget Management** - Dashboard widget configuration
16. **Dashboard Sharing** - Dashboard access and sharing settings

### Usage Statistics (UsageService)
17. **System Usage** - Overall system usage statistics
18. **Feature Usage** - Feature adoption and usage metrics
19. **Storage Analytics** - Storage utilization and trends
20. **License Analytics** - License usage and optimization

### Business Intelligence (BIService)
21. **Data Exports** - `GET /api/{version}/analytics/exports`
22. **Data Warehouse** - Data warehouse integration
23. **ETL Operations** - Extract, Transform, Load operations
24. **Data Pipelines** - Data processing pipeline management

### Custom Analytics (CustomAnalyticsService)
25. **Custom Metrics** - User-defined metrics and KPIs
26. **Custom Reports** - Custom report creation and management
27. **Analytics APIs** - Custom analytics endpoint creation
28. **Data Visualization** - Custom visualization components

### Real-time Analytics (RealtimeService)
29. **Live Metrics** - Real-time system metrics
30. **Event Streaming** - Real-time event data streams
31. **Alert Management** - Analytics-based alerting
32. **Monitoring Dashboards** - Real-time monitoring displays

### 🔬 Testing Strategy
- **Discovery First**: Query analytics metadata to understand available analytics capabilities
- **Safe Operations**: Use read-only analytics retrieval, avoid modifying analytics configurations
- **Intelligent Testing**: Test available analytics endpoints and report generation
- **Non-Destructive**: Ensure analytics tests don't impact system performance or configurations

---

In [1]:
# Essential imports for comprehensive Analytics API testing
import sys
import time
import json
import requests
import tempfile
import os
from typing import Dict, List, Any, Optional
from datetime import datetime, timezone

# VeevaVault imports
sys.path.append('/Users/mp/Documents/Code/VeevaTools/veevatools')
from veevavault.client import VaultClient
from veevavault.services.authentication import AuthenticationService

print("✅ Essential imports loaded for Analytics API testing")

✅ Essential imports loaded for Analytics API testing


## 🔐 Load Test Credentials

Loading secure credentials for VeevaVault API testing.

In [2]:
# Import credentials from secure configuration
try:
    from test_credentials import TEST_VAULTS, DEFAULT_VAULT
    
    # Get the default vault credentials
    vault_config = TEST_VAULTS[DEFAULT_VAULT]
    VAULT_URL = vault_config["URL"]
    VAULT_USERNAME = vault_config["username"] 
    VAULT_PASSWORD = vault_config["password"]
    VAULT_DNS = VAULT_URL.replace("https://", "").replace("http://", "").rstrip("/")
    
    print("✅ Successfully imported credentials from test_credentials.py")
    print(f"🏢 Using vault: {DEFAULT_VAULT}")
    print(f"📡 Vault DNS: {VAULT_DNS}")
    print(f"👤 Username: {VAULT_USERNAME}")
    print(f"🔗 Full URL: {VAULT_URL}")
    
except ImportError as e:
    print("❌ Failed to import credentials!")
    print("Please ensure test_credentials.py exists and contains:")
    print("- TEST_VAULTS dictionary")
    print("- DEFAULT_VAULT setting")
    raise e
except KeyError as e:
    print(f"❌ Configuration error: {e}")
    print("Please check your test_credentials.py configuration")
    raise e

✅ Successfully imported credentials from test_credentials.py
🏢 Using vault: michael_mastermind
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔗 Full URL: https://vv-consulting-michael-mastermind.veevavault.com/


## 🚀 Initialize Enhanced Framework

Setting up the enhanced baseline API testing framework with Analytics-specific functionality.

In [3]:
# Enhanced baseline API testing framework for Analytics operations
from dataclasses import dataclass
from typing import Dict, List, Any, Optional
import time
import uuid

@dataclass
class APITestResult:
    """Standard result structure for API tests"""
    endpoint: str
    method: str
    success: bool
    response_time: float
    status_code: Optional[int] = None
    response_data: Optional[Dict] = None
    error_message: Optional[str] = None
    test_name: Optional[str] = None
    details: Optional[Dict] = None

class EnhancedAnalyticsAPITester:
    """Enhanced baseline API testing framework with Analytics-specific functionality"""
    
    def __init__(self, vault_url: str, username: str, password: str):
        self.vault_url = vault_url.rstrip('/')
        self.vault_dns = vault_url.replace("https://", "").replace("http://", "").rstrip("/")
        self.username = username
        self.password = password
        self.client = None
        self.session_id = None
        self.vault_id = None
        self.results = []
        self.api_version = "v25.2"
        
        # Analytics-specific tracking
        self.discovered_analytics = []
        self.discovered_reports = []
        self.discovered_dashboards = []         # Track available dashboards
        self.analytics_metrics = {}             # Track analytics metrics data
        self.performance_data = {}              # Track performance analytics
        self.usage_statistics = {}              # Track usage statistics
        self.report_results = {}                # Track report execution results
        
        # Initialize VaultClient
        print("🔧 Initializing VeevaVault client...")
        self.client = VaultClient()
        self.client.vaultDNS = self.vault_dns
        self.client.vaultURL = self.vault_url
        self.client.vaultUserName = self.username
        self.client.vaultPassword = self.password
        print("✅ VeevaVault client initialized")
        print(f"🔗 Vault URL: {self.vault_url}")
        print(f"📡 Vault DNS: {self.vault_dns}")
        print(f"👤 Username: {self.username}")
        
        # Set up enhanced framework
        print("🔧 Initializing enhanced test framework...")
        print("✅ Enhanced baseline framework ready")
    
    def authenticate(self) -> bool:
        """Authenticate with the vault and initialize services"""
        try:
            print("🔐 Performing authentication...")
            start_time = time.time()
            
            auth_service = AuthenticationService(self.client)
            response = auth_service.authenticate_with_username_password(
                username=self.username,
                password=self.password
            )
            
            response_time = time.time() - start_time
            
            if response and isinstance(response, dict) and response.get('responseStatus') == 'SUCCESS':
                self.session_id = response.get('sessionId')
                self.vault_id = response.get('vaultId')
                
                # Update client with session
                self.client.sessionId = self.session_id
                self.client.vaultId = self.vault_id
                
                result = APITestResult(
                    endpoint='/api/{version}/auth',
                    method='POST',
                    success=True,
                    response_time=response_time,
                    test_name='Authentication',
                    details={'vault_id': self.vault_id, 'has_session': bool(self.session_id)}
                )
                self.results.append(result)
                
                print(f"✅ Authentication [01]: SUCCESS ({response_time:.2f}s)")
                print(f"\\n🎉 Authentication Successful!")
                print(f"🆔 Session ID: {self.session_id[:20]}...")
                print(f"🏢 Vault ID: {self.vault_id}")
                return True
                
        except Exception as e:
            print(f"❌ Authentication failed: {str(e)}")
            return False
            
        return False
    
    def make_authenticated_request(self, endpoint: str, method: str = 'GET', **kwargs) -> APITestResult:
        """Make an authenticated API request"""
        url = f"{self.vault_url}{endpoint}"
        headers = {
            'Authorization': self.session_id,
            'Accept': 'application/json'
        }
        headers.update(kwargs.get('headers', {}))
        
        start_time = time.time()
        try:
            response = requests.request(method, url, headers=headers, **{k: v for k, v in kwargs.items() if k != 'headers'})
            response_time = time.time() - start_time
            
            return APITestResult(
                endpoint=endpoint,
                method=method,
                success=response.status_code < 400,
                response_time=response_time,
                status_code=response.status_code,
                response_data=response.json() if response.content and response.headers.get('content-type', '').startswith('application/json') else None
            )
        except Exception as e:
            response_time = time.time() - start_time
            return APITestResult(
                endpoint=endpoint,
                method=method,
                success=False,
                response_time=response_time,
                error_message=str(e)
            )
    
    def print_summary(self):
        """Print test summary"""
        if not self.results:
            print("📊 No tests completed yet")
            return
            
        successful_tests = len([r for r in self.results if r.success])
        total_time = sum(r.response_time for r in self.results)
        
        print(f"\\n📊 TEST SUMMARY")
        print("=" * 40)
        print(f"✅ SUCCESS: {successful_tests}")
        if successful_tests < len(self.results):
            print(f"❌ FAILED: {len(self.results) - successful_tests}")
        print(f"⏱️ Total time: {total_time:.2f}s")
        print(f"🔐 Session: {'✅ Active' if self.session_id else '❌ None'}")

# Initialize the enhanced analytics tester
analytics_tester = EnhancedAnalyticsAPITester(VAULT_URL, VAULT_USERNAME, VAULT_PASSWORD)

print("✅ Enhanced Analytics API testing framework initialized")

🔧 Initializing VeevaVault client...
✅ VeevaVault client initialized
🔗 Vault URL: https://vv-consulting-michael-mastermind.veevavault.com
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔧 Initializing enhanced test framework...
✅ Enhanced baseline framework ready
✅ Enhanced Analytics API testing framework initialized


## 🔐 Authentication & Service Initialization

Testing authentication and initializing services for comprehensive analytics operations.

In [4]:
# Test 01: Authentication and Services initialization
print("🔐 Testing authentication and services initialization...")
print("=" * 60)

auth_success = analytics_tester.authenticate()

if auth_success:
    print(f"\\n✅ Session established: {analytics_tester.session_id[:20]}...")
    print(f"✅ Vault ID: {analytics_tester.vault_id}")
    
    # Test basic analytics endpoint availability
    print("\\n📋 Testing basic analytics endpoint availability:")
    
    # Test multiple potential analytics endpoints
    analytics_endpoints = [
        f"/api/{analytics_tester.api_version}/analytics",
        f"/api/{analytics_tester.api_version}/analytics/metrics",
        f"/api/{analytics_tester.api_version}/reports",
        f"/api/{analytics_tester.api_version}/dashboards"
    ]
    
    for endpoint_path in analytics_endpoints:
        try:
            test_result = analytics_tester.make_authenticated_request(endpoint_path, params={'limit': 1})
            endpoint_name = endpoint_path.split('/')[-1].title()
            
            if test_result.success:
                print(f"  ✅ {endpoint_name} endpoint accessible (Status: {test_result.status_code})")
            else:
                print(f"  ❌ {endpoint_name} endpoint failed (Status: {test_result.status_code})")
                
        except Exception as e:
            print(f"  ⚠️ {endpoint_name} endpoint test exception: {e}")
    
else:
    print("❌ Authentication failed - cannot proceed with analytics tests")
    
print("\\n" + "=" * 60)

🔐 Testing authentication and services initialization...
🔐 Performing authentication...
✅ Authentication [01]: SUCCESS (0.36s)
\n🎉 Authentication Successful!
🆔 Session ID: 5DF6BFDAF1956FC4E5D8...
🏢 Vault ID: 92425
\n✅ Session established: 5DF6BFDAF1956FC4E5D8...
✅ Vault ID: 92425
\n📋 Testing basic analytics endpoint availability:
  ❌ Analytics endpoint failed (Status: 404)
  ❌ Metrics endpoint failed (Status: 404)
  ❌ Reports endpoint failed (Status: 404)
  ❌ Dashboards endpoint failed (Status: 404)
\n============================================================


## 📊 Analytics Discovery

Discovering available analytics data in the vault.

In [5]:
# Test 02: Retrieve analytics data
print("📊 Testing analytics discovery...")
print("=" * 60)

try:
    start_time = time.time()
    
    # Attempt to retrieve analytics using direct API call
    endpoint = f"/api/{analytics_tester.api_version}/analytics"
    analytics_params = {
        'params': {
            'limit': 20  # Limit to reduce response size
        }
    }
    
    analytics_result = analytics_tester.make_authenticated_request(endpoint, **analytics_params)
    response_time = time.time() - start_time
    
    if analytics_result.success and analytics_result.response_data:
        analytics_data = analytics_result.response_data.get('data', [])
        analytics_tester.discovered_analytics = analytics_data
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=True,
            response_time=response_time,
            test_name='Retrieve Analytics Data',
            details={
                'total_analytics': len(analytics_data) if isinstance(analytics_data, list) else 1,
                'analytics_type': type(analytics_data).__name__,
                'has_analytics_data': bool(analytics_data),
                'sample_analytics_names': [item.get('name', item.get('metric_name')) for item in analytics_data[:3] if isinstance(item, dict)] if isinstance(analytics_data, list) else []
            }
        )
        analytics_tester.results.append(result)
        
        print(f"✅ Retrieve Analytics Data [02]: SUCCESS ({response_time:.2f}s)")
        
        if isinstance(analytics_data, list) and analytics_data:
            print(f"📊 Found {len(analytics_data)} analytics items")
            
            # Show sample analytics
            print("\\n📋 Sample Analytics:")
            for i, item in enumerate(analytics_data[:5]):
                if isinstance(item, dict):
                    item_name = item.get('name', item.get('metric_name', 'Unknown'))
                    item_type = item.get('type', item.get('metric_type', 'Unknown'))
                    item_value = item.get('value', item.get('metric_value', 'Unknown'))
                    print(f"  {i+1}. {item_name}")
                    print(f"     Type: {item_type}, Value: {item_value}")
                else:
                    print(f"  {i+1}. {item}")
                    
        elif isinstance(analytics_data, dict):
            print("📊 Retrieved analytics structure")
            print(f"📊 Analytics keys: {list(analytics_data.keys())}")
        else:
            print("🔍 No analytics found or unexpected format")
        
    else:
        error_msg = f"API returned {analytics_result.status_code}" if analytics_result.status_code else "Unknown error"
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=False,
            response_time=response_time,
            test_name='Retrieve Analytics Data',
            error_message=error_msg
        )
        analytics_tester.results.append(result)
        print(f"❌ Retrieve Analytics Data [02]: FAILED ({response_time:.2f}s)")
        print(f"   Status Code: {analytics_result.status_code}")
        print(f"   Error: {error_msg}")
        
except Exception as e:
    response_time = time.time() - start_time if 'start_time' in locals() else 0
    result = APITestResult(
        endpoint='/api/{version}/analytics',
        method='GET',
        success=False,
        response_time=response_time,
        test_name='Retrieve Analytics Data',
        error_message=str(e)
    )
    analytics_tester.results.append(result)
    print(f"❌ Retrieve Analytics Data [02]: EXCEPTION ({response_time:.2f}s)")
    print(f"   Exception: {str(e)}")

print("\\n" + "=" * 60)

📊 Testing analytics discovery...
❌ Retrieve Analytics Data [02]: FAILED (0.18s)
   Status Code: 404
   Error: API returned 404
\n============================================================


## 📈 Analytics Metrics

Testing analytics metrics retrieval.

In [6]:
# Test 03: Retrieve analytics metrics
print("📈 Testing analytics metrics retrieval...")
print("=" * 60)

try:
    start_time = time.time()
    
    # Attempt to retrieve analytics metrics
    endpoint = f"/api/{analytics_tester.api_version}/analytics/metrics"
    metrics_result = analytics_tester.make_authenticated_request(endpoint)
    response_time = time.time() - start_time
    
    if metrics_result.success and metrics_result.response_data:
        metrics_data = metrics_result.response_data.get('data', {})
        analytics_tester.analytics_metrics = metrics_data
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=True,
            response_time=response_time,
            test_name='Retrieve Analytics Metrics',
            details={
                'has_metrics_data': bool(metrics_data),
                'metrics_type': type(metrics_data).__name__,
                'metrics_fields': list(metrics_data.keys()) if isinstance(metrics_data, dict) else []
            }
        )
        analytics_tester.results.append(result)
        
        print(f"✅ Retrieve Analytics Metrics [03]: SUCCESS ({response_time:.2f}s)")
        
        if isinstance(metrics_data, dict) and metrics_data:
            # Show metrics information
            metrics_fields = list(metrics_data.keys())
            print(f"\\n📈 Metrics Categories: {len(metrics_fields)}")
            for i, field in enumerate(metrics_fields[:8]):
                field_value = metrics_data.get(field)
                if isinstance(field_value, (list, dict)):
                    field_summary = f"{type(field_value).__name__} with {len(field_value)} items"
                else:
                    field_summary = str(field_value)[:50]
                print(f"  {i+1}. {field}: {field_summary}")
            if len(metrics_fields) > 8:
                print(f"  ... and {len(metrics_fields) - 8} more metrics")
        elif isinstance(metrics_data, list):
            print(f"📈 Found {len(metrics_data)} metric entries")
        else:
            print("🔍 Metrics data retrieved but format unclear")
            
    else:
        error_msg = f"API returned {metrics_result.status_code}" if metrics_result.status_code else "Unknown error"
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=False,
            response_time=response_time,
            test_name='Retrieve Analytics Metrics',
            error_message=error_msg
        )
        analytics_tester.results.append(result)
        print(f"❌ Retrieve Analytics Metrics [03]: FAILED ({response_time:.2f}s)")
        print(f"   Status Code: {metrics_result.status_code}")
        print(f"   Error: {error_msg}")
        
except Exception as e:
    response_time = time.time() - start_time if 'start_time' in locals() else 0
    result = APITestResult(
        endpoint='/api/{version}/analytics/metrics',
        method='GET',
        success=False,
        response_time=response_time,
        test_name='Retrieve Analytics Metrics',
        error_message=str(e)
    )
    analytics_tester.results.append(result)
    print(f"❌ Retrieve Analytics Metrics [03]: EXCEPTION ({response_time:.2f}s)")
    print(f"   Exception: {str(e)}")

print("\\n" + "=" * 60)

📈 Testing analytics metrics retrieval...
❌ Retrieve Analytics Metrics [03]: FAILED (0.18s)
   Status Code: 404
   Error: API returned 404
\n============================================================


## 📋 Reports Discovery

Testing reports discovery and management.

In [7]:
# Test 04: Retrieve reports
print("📋 Testing reports discovery...")
print("=" * 60)

try:
    start_time = time.time()
    
    # Attempt to retrieve reports
    endpoint = f"/api/{analytics_tester.api_version}/reports"
    reports_params = {
        'params': {
            'limit': 10  # Limit to reduce response size
        }
    }
    
    reports_result = analytics_tester.make_authenticated_request(endpoint, **reports_params)
    response_time = time.time() - start_time
    
    if reports_result.success and reports_result.response_data:
        reports_data = reports_result.response_data.get('data', [])
        analytics_tester.discovered_reports = reports_data
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=True,
            response_time=response_time,
            test_name='Retrieve Reports',
            details={
                'total_reports': len(reports_data) if isinstance(reports_data, list) else 1,
                'reports_type': type(reports_data).__name__,
                'has_reports_data': bool(reports_data),
                'sample_report_names': [report.get('name', report.get('report_name')) for report in reports_data[:3] if isinstance(report, dict)] if isinstance(reports_data, list) else []
            }
        )
        analytics_tester.results.append(result)
        
        print(f"✅ Retrieve Reports [04]: SUCCESS ({response_time:.2f}s)")
        
        if isinstance(reports_data, list) and reports_data:
            print(f"📋 Found {len(reports_data)} reports")
            
            # Show sample reports
            print("\\n📋 Sample Reports:")
            for i, report in enumerate(reports_data[:5]):
                if isinstance(report, dict):
                    report_name = report.get('name', report.get('report_name', 'Unknown'))
                    report_type = report.get('type', report.get('report_type', 'Unknown'))
                    report_status = report.get('status', report.get('report_status', 'Unknown'))
                    print(f"  {i+1}. {report_name}")
                    print(f"     Type: {report_type}, Status: {report_status}")
                else:
                    print(f"  {i+1}. {report}")
                    
        elif isinstance(reports_data, dict):
            print("📋 Retrieved reports structure")
            if 'reports' in reports_data:
                report_list = reports_data['reports']
                print(f"📊 Contains {len(report_list) if isinstance(report_list, list) else 'N/A'} reports")
        else:
            print("🔍 No reports found or unexpected format")
            
    else:
        error_msg = f"API returned {reports_result.status_code}" if reports_result.status_code else "Unknown error"
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=False,
            response_time=response_time,
            test_name='Retrieve Reports',
            error_message=error_msg
        )
        analytics_tester.results.append(result)
        print(f"❌ Retrieve Reports [04]: FAILED ({response_time:.2f}s)")
        print(f"   Status Code: {reports_result.status_code}")
        print(f"   Error: {error_msg}")
        
except Exception as e:
    response_time = time.time() - start_time if 'start_time' in locals() else 0
    result = APITestResult(
        endpoint='/api/{version}/reports',
        method='GET',
        success=False,
        response_time=response_time,
        test_name='Retrieve Reports',
        error_message=str(e)
    )
    analytics_tester.results.append(result)
    print(f"❌ Retrieve Reports [04]: EXCEPTION ({response_time:.2f}s)")
    print(f"   Exception: {str(e)}")

print("\\n" + "=" * 60)

📋 Testing reports discovery...
❌ Retrieve Reports [04]: FAILED (0.18s)
   Status Code: 404
   Error: API returned 404
\n============================================================


## 🎛️ Dashboards Discovery

Testing dashboards discovery and management.

In [8]:
# Test 05: Retrieve dashboards
print("🎛️ Testing dashboards discovery...")
print("=" * 60)

try:
    start_time = time.time()
    
    # Attempt to retrieve dashboards
    endpoint = f"/api/{analytics_tester.api_version}/dashboards"
    dashboards_params = {
        'params': {
            'limit': 10  # Limit to reduce response size
        }
    }
    
    dashboards_result = analytics_tester.make_authenticated_request(endpoint, **dashboards_params)
    response_time = time.time() - start_time
    
    if dashboards_result.success and dashboards_result.response_data:
        dashboards_data = dashboards_result.response_data.get('data', [])
        analytics_tester.discovered_dashboards = dashboards_data
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=True,
            response_time=response_time,
            test_name='Retrieve Dashboards',
            details={
                'total_dashboards': len(dashboards_data) if isinstance(dashboards_data, list) else 1,
                'dashboards_type': type(dashboards_data).__name__,
                'has_dashboards_data': bool(dashboards_data),
                'sample_dashboard_names': [dashboard.get('name', dashboard.get('dashboard_name')) for dashboard in dashboards_data[:3] if isinstance(dashboard, dict)] if isinstance(dashboards_data, list) else []
            }
        )
        analytics_tester.results.append(result)
        
        print(f"✅ Retrieve Dashboards [05]: SUCCESS ({response_time:.2f}s)")
        
        if isinstance(dashboards_data, list) and dashboards_data:
            print(f"🎛️ Found {len(dashboards_data)} dashboards")
            
            # Show sample dashboards
            print("\\n📋 Sample Dashboards:")
            for i, dashboard in enumerate(dashboards_data[:5]):
                if isinstance(dashboard, dict):
                    dashboard_name = dashboard.get('name', dashboard.get('dashboard_name', 'Unknown'))
                    dashboard_type = dashboard.get('type', dashboard.get('dashboard_type', 'Unknown'))
                    dashboard_status = dashboard.get('status', dashboard.get('dashboard_status', 'Unknown'))
                    print(f"  {i+1}. {dashboard_name}")
                    print(f"     Type: {dashboard_type}, Status: {dashboard_status}")
                else:
                    print(f"  {i+1}. {dashboard}")
                    
        elif isinstance(dashboards_data, dict):
            print("🎛️ Retrieved dashboards structure")
            if 'dashboards' in dashboards_data:
                dashboard_list = dashboards_data['dashboards']
                print(f"📊 Contains {len(dashboard_list) if isinstance(dashboard_list, list) else 'N/A'} dashboards")
        else:
            print("🔍 No dashboards found or unexpected format")
            
    else:
        error_msg = f"API returned {dashboards_result.status_code}" if dashboards_result.status_code else "Unknown error"
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=False,
            response_time=response_time,
            test_name='Retrieve Dashboards',
            error_message=error_msg
        )
        analytics_tester.results.append(result)
        print(f"❌ Retrieve Dashboards [05]: FAILED ({response_time:.2f}s)")
        print(f"   Status Code: {dashboards_result.status_code}")
        print(f"   Error: {error_msg}")
        
except Exception as e:
    response_time = time.time() - start_time if 'start_time' in locals() else 0
    result = APITestResult(
        endpoint='/api/{version}/dashboards',
        method='GET',
        success=False,
        response_time=response_time,
        test_name='Retrieve Dashboards',
        error_message=str(e)
    )
    analytics_tester.results.append(result)
    print(f"❌ Retrieve Dashboards [05]: EXCEPTION ({response_time:.2f}s)")
    print(f"   Exception: {str(e)}")

print("\\n" + "=" * 60)

🎛️ Testing dashboards discovery...
❌ Retrieve Dashboards [05]: FAILED (0.17s)
   Status Code: 404
   Error: API returned 404
\n============================================================


## 🎯 Test Summary & Cleanup

Comprehensive results analysis for analytics testing (all operations are read-only and safe).

In [9]:
# Comprehensive Test Summary and Cleanup
print("🎯 ANALYTICS API TESTING - FINAL SUMMARY")
print("=" * 60)

# Print comprehensive summary
analytics_tester.print_summary()

# Detailed results analysis
print("\\n📊 DETAILED RESULTS")
print("-" * 40)

success_count = 0
total_time = 0

for i, result in enumerate(analytics_tester.results, 1):
    status = "✅ PASS" if result.success else "❌ FAIL"
    test_name = result.test_name or f"Test {i}"
    
    print(f"{status} [{i:02d}]: {test_name} ({result.response_time:.2f}s)")
    
    if result.success:
        success_count += 1
        
    total_time += result.response_time
    
    # Show additional details for interesting results
    if result.details:
        if 'total_analytics' in result.details:
            print(f"        📊 Analytics Found: {result.details['total_analytics']}")
        if 'total_reports' in result.details:
            print(f"        📋 Reports Found: {result.details['total_reports']}")
        if 'total_dashboards' in result.details:
            print(f"        🎛️ Dashboards Found: {result.details['total_dashboards']}")
        if 'metrics_fields' in result.details and result.details['metrics_fields']:
            print(f"        📈 Metrics Categories: {len(result.details['metrics_fields'])}")
        if 'vault_id' in result.details:
            print(f"        🏢 Vault ID: {result.details['vault_id']}")
    
    if not result.success and result.error_message:
        print(f"        ❌ Error: {result.error_message}")

# Final statistics
print("\\n🏆 FINAL STATISTICS")
print("-" * 40)
print(f"📈 Success Rate: {success_count}/{len(analytics_tester.results)} ({(success_count/len(analytics_tester.results)*100):.1f}%)")
print(f"⏱️ Total Execution Time: {total_time:.2f}s")
print(f"📊 Analytics Discovered: {len(analytics_tester.discovered_analytics)}")
print(f"📋 Reports Discovered: {len(analytics_tester.discovered_reports)}")
print(f"🎛️ Dashboards Discovered: {len(analytics_tester.discovered_dashboards)}")
print(f"📈 Analytics Metrics Available: {bool(analytics_tester.analytics_metrics)}")
print(f"📊 Performance Data Available: {bool(analytics_tester.performance_data)}")

# Vault state summary
print("\\n🏢 VAULT STATE")
print("-" * 40)
print(f"🔗 Connected to: {analytics_tester.vault_url}")
print(f"🆔 Vault ID: {analytics_tester.vault_id}")
print(f"👤 User: {analytics_tester.username}")
print(f"🔐 Session: {'Active' if analytics_tester.session_id else 'None'}")

# API coverage summary
tested_endpoints = [
    "Authentication",
    "Retrieve Analytics Data", 
    "Retrieve Analytics Metrics",
    "Retrieve Reports",
    "Retrieve Dashboards"
]

print("\\n🎯 API COVERAGE")
print("-" * 40)
for i, endpoint in enumerate(tested_endpoints, 1):
    status = "✅" if i <= len(analytics_tester.results) else "⏸️"
    print(f"{status} {endpoint}")

print("\\n✅ Analytics API testing completed!")
print("📋 All operations were read-only and analytics-safe")
print("📊 No analytics configurations or data modified - vault integrity preserved")

# Export summary for next section
analytics_summary = {
    'section': '15_analytics',
    'total_tests': len(analytics_tester.results),
    'successful_tests': success_count,
    'total_time': total_time,
    'success_rate': (success_count/len(analytics_tester.results)*100) if analytics_tester.results else 0,
    'discovered_analytics': len(analytics_tester.discovered_analytics),
    'discovered_reports': len(analytics_tester.discovered_reports),
    'discovered_dashboards': len(analytics_tester.discovered_dashboards),
    'analytics_metrics_available': bool(analytics_tester.analytics_metrics),
    'performance_data_available': bool(analytics_tester.performance_data),
    'vault_id': analytics_tester.vault_id,
    'session_active': bool(analytics_tester.session_id)
}

print(f"\\n📦 Section summary exported for continuation: {analytics_summary['success_rate']:.1f}% success rate")
print("=" * 60)

🎯 ANALYTICS API TESTING - FINAL SUMMARY
\n📊 TEST SUMMARY
✅ SUCCESS: 1
❌ FAILED: 4
⏱️ Total time: 1.07s
🔐 Session: ✅ Active
\n📊 DETAILED RESULTS
----------------------------------------
✅ PASS [01]: Authentication (0.36s)
        🏢 Vault ID: 92425
❌ FAIL [02]: Retrieve Analytics Data (0.18s)
        ❌ Error: API returned 404
❌ FAIL [03]: Retrieve Analytics Metrics (0.18s)
        ❌ Error: API returned 404
❌ FAIL [04]: Retrieve Reports (0.18s)
        ❌ Error: API returned 404
❌ FAIL [05]: Retrieve Dashboards (0.17s)
        ❌ Error: API returned 404
\n🏆 FINAL STATISTICS
----------------------------------------
📈 Success Rate: 1/5 (20.0%)
⏱️ Total Execution Time: 1.07s
📊 Analytics Discovered: 0
📋 Reports Discovered: 0
🎛️ Dashboards Discovered: 0
📈 Analytics Metrics Available: False
📊 Performance Data Available: False
\n🏢 VAULT STATE
----------------------------------------
🔗 Connected to: https://vv-consulting-michael-mastermind.veevavault.com
🆔 Vault ID: 92425
👤 User: veevatools@vv-cons